# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおいて、Qwen2-7b-itモデルを使用したエージェントを作成するための実装を示しています。Notebookの内容は、具体的にはエージェントが「20の質問ゲーム」において質問と推測を行い、適切に応答する仕組みを構築することに取り組んでいます。

### 問題の概要
本エージェントは、言語モデルを使用して、ユーザーから提示されたキーワードに関連する質問を生成し、それに対して「はい」または「いいえ」で回答する役割を果たします。また、エージェントは求められた際にキーワードを推測する能力も持っています。このようにして、最小限の質問回数でキーワードを特定することが求められます。

### 使用される手法とライブラリ
1. **ライブラリのインストール**:
   - `bitsandbytes` や `accelerate` を用いてCUDAのメモリ効率を向上させ、モデルの実行効率を高めています。
   - `transformers` ライブラリを使用して、事前訓練済みモデルとトークナイザーを便利に読み込んでいます。

2. **モデルの設定**:
   - `AutoTokenizer` でトークナイザーを初期化し、`AutoModelForCausalLM` を使用してQwen2-7b-itモデルを読み込みます。モデルは量子化設定を利用してメモリ使用量を削減しつつ、効率よく動作するようにしています。

3. **エージェントの実装**:
   - `Robot` クラスを定義し、質問と推測を行うためのメソッド (`asker`, `answerer`) を実装しています。これにより、質問を生成したり、回答を行ったりする機能を持ったエージェントを構築しています。
   - `agent` 関数でエージェントが観察情報に基づき適切な動作を取るようになっています。

4. **シミュレーション機能**:
   - コメントアウトされたコードブロックがあり、実際のゲームをシミュレーションするための環境を設定することが可能です。この部分では、観察者のクラスを定義し、ゲームのルールに則ってエージェントのアクションを試行することができます。

このNotebook全体として、Qwen2-7b-itモデルを活用し、「20の質問ゲーム」を成功裏にプレイするためのAIエージェントの実装と、そのための環境設定、シミュレーション機能を包含しています。

---


# 用語概説 
以下は、提供されたJupyter Notebookに関連する専門用語の簡単な解説です。初心者がつまずきそうなものに焦点を当てていますが、一般的に知られている用語は省いています。

### 専門用語解説

1. **メモリ効率の良いSDP (Stochastic Dynamic Programming)**:
   - メモリ効率を考慮した動的プログラミングの手法で、メモリ消費を最小限に抑えながら計算を行うことを指します。特に大きなモデルを扱う際に役立ちます。

2. **量子化 (Quantization)**:
   - モデルのサイズを削減し、推論を高速化するために、モデルの重みを小さなビット数で表現する手法です。これにより、メモリ使用量が低減し、計算速度が向上します。特に「4-bit量子化」は一般的な手法です。

3. **BitsAndBytesConfig**:
   - 量子化のための設定を行うためのクラスで、どのようにモデルを量子化するかを指定します。パラメータには、量子化のタイプや計算時のデータ型が含まれます。

4. **Pipeline**:
   - Hugging Face Transformersライブラリにおいて、特定のタスク（例えばテキスト生成、感情分析など）を行うための簡潔なインターフェースです。モデルとトークナイザーを組み合わせて使うことができます。

5. **チャットテンプレート (Chat Template)**:
   - 生成したいテキストの形式や構造を示すために使用するフレームワークです。具体的には、システムの指示やユーザーとの対話履歴を含むことがあります。

6. **レギュライズ (Regularization)**:
   - モデルの複雑さを減少させ、過学習を防ぐために使用される手法です。特に、ニューラルネットワークでは、ドロップアウトやL2正則化などがあります。

7. **デヴ (Dev)**:
   - 開発環境を指し、モデルのトレーニングや動作確認、デバッグを行うための設定や仮想環境です。

8. **エージェント (Agent)**:
   - ゲームやシミュレーションにおいて、特定の役割を持ち、自律的に行動するプログラムやモデルです。このノートブックでは、質問者と回答者のエージェントが実装されています。

9. **オブザベーション (Observation)**:
   - エージェントが環境から得られる情報や状態を指します。このクラスは、ゲームの進行に必要な情報（質問、回答、キーワードなど）を管理します。

10. **サンプリング (Sampling)**:
    - モデルから生成される出力を選択する際の手法です。通常、確率的な方法に基づいており、生成するテキストの多様性を向上させます。たとえば、`do_sample`オプションによって制御されます。

この解説リストは、機械学習や深層学習において一般的に使われる用語以外の、特定の技術や手法に焦点を当てたものです。

---


# Qwen2-7b-it
参考のために、[LLMリーダーボード](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard)で良い成績を収めているQwen2-7b-itを用いて作成されたエージェントを共有します。

In [ ]:
%%bash
# /kaggle/working/submissionというディレクトリを作成します。
mkdir -p /kaggle/working/submission
# bitsandbytesとaccelerateというライブラリをインストールします。
pip install bitsandbytes accelerate
# transformersライブラリを最新バージョンにアップグレードします。
pip install -U transformers

In [ ]:
%%writefile -a submission/main.py

# submission/main.pyというファイルに追記します

import os
import torch
import random
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline

# CUDAを使用したメモリ効率の良いSDPを有効にします
torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
# Kaggleエージェントのパスが存在するか確認します
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")
else:
    model_id = "/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1"

# トークナイザーをモデルIDから読み込みます
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 量子化設定を定義します
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
# モデルを読み込む際の設定をします
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
)
# テキスト生成のためのパイプラインを作成します
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def parse_response(response):
    # レスポンスから質問を抽出します
    match = re.search(".+?\?", response.replace('*', ''))
    if match is None:
        question = 'キーワードは「s」という文字で始まりますか？'
    else:
        question = match.group()
    return question

def generate_answer(chat_template):
    # 与えられたチャットテンプレートに基づいて回答を生成します
    output = pipe(
        chat_template,
        max_new_tokens=32,
        do_sample=False,
#         temperature=0.01,
#         top_p=0.1,
#         top_k=1,
        return_full_text=False,
    )[0]["generated_text"]
    # 出力から<end_of_turn>を削除します
    output = re.sub('<end_of_turn>', '', output)
    return output

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        # モードに応じた処理を行います
        assert mode in ["asking", "guessing", "answering"], "modeはasking, answering, guessingのいずれかでなければなりません"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "no"
        if mode == "guessing":
            output = self.asker(obs)
        return output

    def asker(self, obs):
        # 質問者としての役割を果たします
        sys_prompt = """
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。
        このゲームでは、回答者がキーワードを考え、質問者の「はい」または「いいえ」の質問に答えます。
        キーワードは具体的な「物」または「場所」です。
        """
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            20の質問を始めましょう。あなたは質問者の役割を果たします。
            「はい」または「いいえ」で答えられる質問をしてください。
            例として、以下のように進行します：
            例：キーワードは「モロッコ」であるとします：
            <assistant: それは都市ですか？
            user: いいえ
            assistant: それは国ですか？
            user: はい
            assistant: アフリカにありますか？
            user: はい
            assistant: その国名はmで始まりますか？
            user: はい
            assistant: モロッコですか？
            user: はい。>
            """

            chat_template = f"""<start_of_turn>system\n{ask_prompt}\n"""

            if len(obs.questions)>=1:
                chat_template += "これまでの会話の履歴です：\n"
                chat_template += "<\n"
                for q, a in zip(obs.questions, obs.answers):
                    chat_template += f"assistant:\n{q}\n"
                    chat_template += f"user:\n{a}\n"
                chat_template += ">\n"
                    
            chat_template += """
            ユーザーが選んだ単語について、最初の質問をしてください！
            短く簡潔に、質問は一つだけで、余計な言葉は使わないでください！<end_of_turn>
            <start_of_turn>assistant\n
            """
#             print(chat_template)
            output = generate_answer(chat_template) 
            output = parse_response(output)
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):
                conv += f"""質問:\n{q}\n回答:\n{a}\n"""
            guess_prompt =  sys_prompt + f"""
            あなたの役割は質問と回答に基づいてキーワードを予測することです。
            進行の例は以下の通りです：
            例：
            <質問: それは都市ですか？
            回答: いいえ
            質問: それは国ですか？
            回答: はい
            質問: アフリカにありますか？
            回答: はい
            質問: その国名はmで始まりますか？
            回答: はい
            あなた: モロッコ>
            
            現在のゲームの状況は以下の通りです:\n{conv}
            この会話に基づいて、単語を予測してください。余計な言葉は使わずにください<end_of_turn>
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}\n"""
            chat_template += "<start_of_turn>あなた:\n"
#             print(chat_template)
            output = generate_answer(chat_template)        
        return output
    
    def answerer(self, obs):
        # 質問に対する回答を提供します
        ask_prompt = f"""
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者の「はい」または「いいえ」の質問に答えます。
        進行の例は以下の通りです：
        例：キーワードが「モロッコ」の場合
        <user: それは場所ですか？
        あなた: はい
        user: ヨーロッパにありますか？
        あなた: いいえ
        user: アフリカにありますか？
        あなた: はい
        user: その国名はmで始まりますか？
        あなた: はい
        user: モロッコですか？
        あなた: はい。>
        「{obs.keyword}」に関して、以下の質問には「はい」または「いいえ」で答えてください：
        {obs.questions[-1]}
        """
        chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>アシスタント\n"
        output = generate_answer(chat_template)
        return output

# ロボットのインスタンスを作成します
robot = Robot()

def agent(obs, cfg):
    # エージェントが行う処理
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:
        response = "はい"
        
    return response

In [ ]:
# pigzとpvをインストールします。出力は表示しないようにします。
!apt install pigz pv > /dev/null

In [ ]:
# pigzを使用して高速に圧縮し、pvで進行状況を表示しながらsubmission.tar.gzというアーカイブを作成します
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/input/qwen2/transformers/qwen2-7b-instruct . -C /kaggle/working/submission .

In [ ]:
# tar.gzファイルの中身を確認するためのコードです

import tarfile
# submission.tar.gzファイルを開きます
tar = tarfile.open("/kaggle/working/submission.tar.gz")
# アーカイブ内のファイル名をすべて印刷します
for file in tar.getmembers():
    print(file.name)

# シミュレーション
エージェントを提出する前にシミュレーションを行いたい場合は、以下のコメントを解除して実行してください。

In [ ]:
# keywords_local.pyというファイルを指定したURLからダウンロードします
# !wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
# モジュールをインポートします
# import json
# import pandas as pd
# submission.mainモジュールからエージェントをインポートします
# from submission.main import agent
# keywords_localからKEYWORDS_JSONをインポートします
# from keywords_local import KEYWORDS_JSON

# 観察用のクラスを定義します
class Observation:
    def __init__(self):
        self.step = 0  # ステップカウンタ
        self.role = "guesser"  # ロールは「推測者」
        self.turnType = "ask"  # ゲームのターンタイプは「質問」
        self.keyword = "Japan"  # キーワードの初期値
        self.category = "country"  # カテゴリーの初期値
        self.questions = []  # 質問のリスト
        self.answers = []  # 回答のリスト
        self.guesses = []  # 推測のリスト
        
# KEYWORDS_JSONを使ってキーワードのデータフレームを作成する関数を定義します
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSONデータをロードします

    keyword_list = []  # キーワードのリスト
    category_list = []  # カテゴリーのリスト
    alts_list = []  # 代替案のリスト

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']  # キーワードを取得
            keyword_list.append(keyword)  # リストに追加
            category_list.append(json_data[i]['category'])  # カテゴリーを追加
            alts_list.append(json_data[i]['words'][j]['alts'])  # 代替案を追加

    # データフレームを作成します
    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd  # 作成したデータフレームを返します
    
# KEYWORDS_JSONを基にキーワードのデータフレームを作成します
# keywords_df = create_keyword_df(KEYWORDS_JSON)

In [ ]:
# 観察インスタンスを作成します
# obs = Observation()
# コンフィグは "_" で初期化します
# cfg = "_"

# keywords_dfからサンプルを取得します
# sample_df = keywords_df.sample()
# 観察にキーワードとカテゴリーを設定します
# obs.keyword = sample_df["keyword"].values[0]
# obs.category = sample_df["category"].values[0]
# 代替案リストを取得します
# alts_list = sample_df["alts"].values[0]
# キーワードを代替案リストに追加します
# alts_list.append(obs.keyword)

# 選ばれたキーワードを表示します
# print(f"keyword:{obs.keyword}")

# 20ラウンドまでのゲームをシミュレーションします
# for round in range(20):
#     obs.step = round+1  # ラウンド数を更新
    
#     obs.role = "guesser"  # ロールを「推測者」に設定
#     obs.turnType = "ask"  # ターンタイプを「質問」に設定
#     question = agent(obs, cfg)  # エージェントに質問をさせます
#     obs.questions.append(question)  # 質問を記録します
    
#     obs.role = "answerer"  # ロールを「回答者」に設定
#     obs.turnType = "answer"  # ターンタイプを「回答」に設定
#     answer = agent(obs, cfg)  # エージェントに回答をさせます
#     obs.answers.append(answer)  # 回答を記録します
    
#     obs.role = "guesser"  # ロールを「推測者」に戻します
#     obs.turnType = "guess"  # ターンタイプを「推測」に設定
#     guess = agent(obs, cfg)  # エージェントに推測をさせます
#     obs.guesses.append(guess)  # 推測を記録します
    
#     # ラウンドの進行状況を表示します
#     print(f"round: {round+1}")
#     print(f"question: {question}")
#     print(f"answer: {answer}")
#     print(f"guess: {guess}")
    
#     # 推測が代替案リストに含まれているかをチェックします
#     if guess in alts_list:
#         print("勝ちました!!")  # 勝利した場合のメッセージ
#         break